In [1]:
import pandas as pd

In [3]:
sample_df = pd.read_csv("./sample_Data/product_name_sample_Data.csv")
sample_df.head()

,product_name,description
0,草本花香洗髮露 600毫升,揉合100%日本培植的有機草本植物，令髮絲彷如重生，令變得柔滑清爽，氛芳花香，令你彷如置身大...
1,Voost 運動水樽,VOOST MUG
2,完美遮瑕筆306 (1.5ml),質地柔亮潤澤，遮瑕的同時去除暗沉，作為 highlight 使用能提亮妝容<BR><BR>獨...
3,高效防脫增生洗髮液 150毫升,ANTI HAIR LOSS SHAMP
4,ISOTONIC 運動水溶片青檸檬味十片裝,幫助人體代謝碳水化合物、脂肪和蛋白質; 快速補充水份與電解質，促進神經肌肉傳導; 有助維持肌...


In [9]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32372 entries, 0 to 32371
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  32361 non-null  object
 1   description   31501 non-null  object
dtypes: object(2)
memory usage: 505.9+ KB


In [11]:
sample_df = sample_df.fillna("*")

In [12]:
product_name_df = sample_df[["product_name"]]
description_df = sample_df[["description"]]

product_name_df.columns = ["doc"]
description_df.columns = ["doc"]

long_df = pd.concat([product_name_df, description_df])
long_df.head()

,doc
0,草本花香洗髮露 600毫升
1,Voost 運動水樽
2,完美遮瑕筆306 (1.5ml)
3,高效防脫增生洗髮液 150毫升
4,ISOTONIC 運動水溶片青檸檬味十片裝


In [13]:
long_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64744 entries, 0 to 32371
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doc     64744 non-null  object
dtypes: object(1)
memory usage: 1011.6+ KB


In [15]:
import re


def clean_text(text:str) -> str:
    # 去除 HTML 標籤
    text = re.sub(r'<.*?>', '#', text)
    # 去除非字母字符
    text = re.sub(r'[^a-zA-Z\u4e00-\u9fa5]', '#', text)
    # 轉換為小寫（僅針對英文）
    text = text.lower()

    return text

In [17]:
long_df["doc"] = long_df["doc"].apply(clean_text)
long_df.head()

,doc
0,草本花香洗髮露####毫升
1,voost#運動水樽
2,完美遮瑕筆########ml#
3,高效防脫增生洗髮液####毫升
4,isotonic#運動水溶片青檸檬味十片裝


In [18]:
def do_n_gram(doc: str, n: int=2) -> list[str]:
    return [doc[i: i+n] for i in range(len(doc) - (n+1))]

In [ ]:
colunm_name = ["bigram", ]
long_df["count"] = 1

In [19]:
for i in range(2, 5):
    long_df[f"{i}_gram"] = long_df["doc"].apply(lambda x: do_n_gram(x, i))

In [27]:
long_df.head()

,doc,2_gram,3_gram,4_gram,count
0,草本花香洗髮露####毫升,"[草本, 本花, 花香, 香洗, 洗髮, 髮露, 露#, ##, ##, ##]","[草本花, 本花香, 花香洗, 香洗髮, 洗髮露, 髮露#, 露##, ###, ###]","[草本花香, 本花香洗, 花香洗髮, 香洗髮露, 洗髮露#, 髮露##, 露###, ####]",1
1,voost#運動水樽,"[vo, oo, os, st, t#, #運, 運動]","[voo, oos, ost, st#, t#運, #運動]","[voos, oost, ost#, st#運, t#運動]",1
2,完美遮瑕筆########ml#,"[完美, 美遮, 遮瑕, 瑕筆, 筆#, ##, ##, ##, ##, ##, ##, #...","[完美遮, 美遮瑕, 遮瑕筆, 瑕筆#, 筆##, ###, ###, ###, ###, ...","[完美遮瑕, 美遮瑕筆, 遮瑕筆#, 瑕筆##, 筆###, ####, ####, ###...",1
3,高效防脫增生洗髮液####毫升,"[高效, 效防, 防脫, 脫增, 增生, 生洗, 洗髮, 髮液, 液#, ##, ##, ##]","[高效防, 效防脫, 防脫增, 脫增生, 增生洗, 生洗髮, 洗髮液, 髮液#, 液##, ...","[高效防脫, 效防脫增, 防脫增生, 脫增生洗, 增生洗髮, 生洗髮液, 洗髮液#, 髮液#...",1
4,isotonic#運動水溶片青檸檬味十片裝,"[is, so, ot, to, on, ni, ic, c#, #運, 運動, 動水, 水...","[iso, sot, oto, ton, oni, nic, ic#, c#運, #運動, ...","[isot, soto, oton, toni, onic, nic#, ic#運, c#運...",1


In [68]:
bigram_df = long_df[["2_gram", "count"]].explode("2_gram").groupby("2_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False)
bigram_df = bigram_df[~bigram_df["2_gram"].str.contains("#")].reset_index(drop=True)
bigram_df.head(20)

,2_gram,count
0,肌膚,14632
1,配方,6326
2,in,6148
3,保濕,5789
4,使用,5311
5,天然,4904
6,精華,4885
7,有效,4742
8,成分,4163
9,er,4067


In [69]:
trigram_df = long_df[["3_gram", "count"]].explode("3_gram").groupby("3_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False)
trigram_df = trigram_df[~trigram_df["3_gram"].str.contains("#")].reset_index(drop=True)
trigram_df.head(20)

,3_gram,count
0,維他命,2794
1,屈臣氏,1802
2,ing,1723
3,透明質,1111
4,明質酸,1106
5,無添加,1098
6,spf,1089
7,防腐劑,1084
8,shi,1083
9,ine,1030


In [70]:
fourgram_df = long_df[["4_gram", "count"]].explode("4_gram").groupby("4_gram", as_index=False).agg({"count": "sum"}).sort_values("count", ascending=False)
fourgram_df = fourgram_df[~fourgram_df["4_gram"].str.contains("#")].reset_index(drop=True)
fourgram_df.head(20)

,4_gram,count
0,透明質酸,1104
1,shin,895
2,hing,861
3,膠原蛋白,818
4,diva,798
5,ashi,797
6,dash,796
7,全新造型,746
8,品牌全新,744
9,指甲品牌,744


In [112]:
bigram_df["2_gram"].shape

(153944,)

In [107]:
bigram_df["2_gram"][bigram_df.iloc[:, 1] > 1].shape

(98001,)

In [105]:
# def find_matches(ngram1, ngram2):
#     return ngram2[(ngram2.iloc[:, 1] > 1) & (ngram2.iloc[:, 0].str.contains(ngram1))].iloc[:, 0].to_list()

In [171]:
def find_matches(ngram1, ngram2):
    pattern = re.compile(ngram1)
    filtered_df = ngram2[ngram2.iloc[:, 1] > 1]
    matches = filtered_df[filtered_df.iloc[:, 0].apply(lambda x: bool(pattern.search(x)))]
    return [(row.iloc[0], row.iloc[1]) for _, row in matches.iterrows()]

In [172]:
bi_tri_df = bigram_df.copy()

bi_tri_df["compare"] = bi_tri_df["2_gram"][bi_tri_df.iloc[:, 1] > 1].apply(lambda x: find_matches(x, trigram_df))
bi_tri_df["group_count"] = bi_tri_df["compare"].apply(lambda x: len(x))
bi_tri_df["keep_2_gram"] = bi_tri_df["group_count"].apply(lambda x: True if x>100 else False)
bi_tri_df.to_csv("./n_gram_processed/bi_tri_df.csv")
bi_tri_df.head(20)

,2_gram,count,compare,group_count,keep_2_gram
0,肌膚,14632,"[(令肌膚, 916), (為肌膚, 767), (肌膚的, 656), (的肌膚, 583...",649.0,True
1,配方,6326,"[(配方奶, 315), (配方含, 164), (水配方, 163), (的配方, 152...",506.0,True
2,in,6148,"[(ing, 1723), (ine, 1030), (hin, 1012), (lin, ...",86.0,False
3,保濕,5789,"[(效保濕, 550), (保濕成, 440), (潤保濕, 273), (保濕精, 264...",418.0,True
4,使用,5311,"[(續使用, 233), (使用後, 227), (使用方, 225), (使用時, 179...",492.0,True
5,天然,4904,"[(的天然, 482), (天然成, 354), (含天然, 326), (天然植, 242...",450.0,True
6,精華,4885,"[(精華液, 366), (取精華, 288), (濕精華, 263), (精華及, 253...",446.0,True
7,有效,4742,"[(能有效, 578), (有效減, 180), (有效舒, 180), (有效清, 174...",301.0,True
8,成分,4163,"[(濕成分, 327), (性成分, 225), (然成分, 205), (養成分, 195...",358.0,True
9,er,4067,"[(ter, 587), (per, 585), (era, 395), (der, 367...",64.0,False


In [193]:
tri_four_df = trigram_df.copy()

tri_four_df["compare"] = tri_four_df["3_gram"][tri_four_df.iloc[:, 1] > 1][:100].apply(lambda x: find_matches(x, fourgram_df))
tri_four_df["group_count"] = tri_four_df["compare"][:100].apply(lambda x: len(x))
tri_four_df["keep_3_gram"] = tri_four_df["group_count"][:100].apply(lambda x: True if x>100 else False)
tri_four_df.head(20)

,3_gram,count,compare,group_count,keep_3_gram
0,維他命,2794,"[(維他命c, 660), (維他命e, 488), (維他命b, 417), (及維他命,...",157.0,True
1,屈臣氏,1802,"[(屈臣氏骨, 253), (屈臣氏蒸, 58), (屈臣氏燕, 57), (送屈臣氏, 4...",194.0,True
2,ing,1723,"[(hing, 861), (ting, 185), (ning, 101), (sing,...",30.0,False
3,透明質,1111,"[(透明質酸, 1104), (含透明質, 98), (及透明質, 79), (子透明質, ...",66.0,False
4,明質酸,1106,"[(透明質酸, 1104), (明質酸及, 126), (明質酸鈉, 47), (明質酸保,...",56.0,False
5,無添加,1098,"[(無添加糖, 92), (無添加螢, 88), (無添加防, 83), (大無添加, 74...",75.0,False
6,spf,1089,"[(霜spf, 120), (液spf, 51), (乳spf, 39), (有spf, 2...",26.0,False
7,防腐劑,1084,"[(含防腐劑, 167), (類防腐劑, 157), (加防腐劑, 121), (n防腐劑,...",33.0,False
8,shi,1083,"[(shin, 895), (ashi, 797), (ushi, 123), (shio,...",15.0,False
9,ine,1030,"[(line, 557), (iner, 106), (hine, 82), (mine, ...",45.0,False
